In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Introduction
* Raw CSV loaded and lightly processed. Output: two column csv columns, property address, flat tag
* Data labelled in programmatic. Output: json file of entities.
* Data programmatic output json cleaned ordered and overlaps removed. Output: json file
* **Clean json converted to dataframe and multi-addresses expanded.** Output: CSV
* Count and locate addresses
* Create address matcher and match businesses
* Classify address types

This notebook is used so that I can create the code necessary to expand the addresses so that a single property/dwelling is a single line. This means the data will follow the tidy data principles of one observation per line where an observation is what is commonly thought of as a property.

In [9]:
import json
import pandas as pd
import re
from helper_functions import *
import numpy as np

In [11]:

with open('/tf/empty_homes_data/full_dataset_no_overlaps.json', "r") as read_file:
    all_entities_json = json.load(read_file)


In [5]:
all_entities_json

{'datapoints': [{'programmatic': {'results': [{'start': 27,
      'end': 39,
      'text': 'nel pan lane',
      'labelId': '12',
      'label': 'street_name',
      'labellingFunctionId': 23,
      'groundTruthId': None},
     {'start': 0,
      'end': 25,
      'text': 'westleigh lodge care home',
      'labelId': '10',
      'label': 'building_name',
      'labellingFunctionId': 30,
      'groundTruthId': None},
     {'start': 41,
      'end': 47,
      'text': 'leigh ',
      'labelId': '3',
      'label': 'city',
      'labellingFunctionId': 12,
      'groundTruthId': None},
     {'start': 27,
      'end': 39,
      'text': 'nel pan lane',
      'labelId': '12',
      'label': 'street_name',
      'labellingFunctionId': 24,
      'groundTruthId': None},
     {'start': 48,
      'end': 55,
      'text': 'wn7 5jt',
      'labelId': '11',
      'label': 'postcode',
      'labellingFunctionId': 36,
      'groundTruthId': None}],
    'aggregateResults': [],
    'groundTruths': []},
   

In [15]:

all_entities = pd.json_normalize(
    all_entities_json["datapoints"],
    record_path=["programmatic", "results"],
    meta=["data", "id"],
    #record_prefix="result_stuff.",
    meta_prefix="data_stuff.",
    errors="ignore",
)

all_entities["label_text"] = all_entities["data_stuff.data"].map(lambda x: x["text"])
all_entities = all_entities.rename(columns = {'data_stuff.id':'datapoint_id'})


all_entities = all_entities.sort_values(['datapoint_id', 'start'])

In [17]:
all_entities

,start,end,text,labelId,label,datapoint_id,label_text,label_id_count
1,0,25,westleigh lodge care home,10,building_name,0,"westleigh lodge care home, nel pan lane, leigh...",0
0,27,39,nel pan lane,12,street_name,0,"westleigh lodge care home, nel pan lane, leigh...",0
3,27,39,nel pan lane,12,street_name,0,"westleigh lodge care home, nel pan lane, leigh...",1
2,41,47,leigh,3,city,0,"westleigh lodge care home, nel pan lane, leigh...",0
4,48,55,wn7 5jt,11,postcode,0,"westleigh lodge care home, nel pan lane, leigh...",0
...,...,...,...,...,...,...,...,...
439450,44,45,3,13,street_number,94087,"storage 17, discovery dock apartments east, 3 ...",0
439449,46,63,south quay square,12,street_name,94087,"storage 17, discovery dock apartments east, 3 ...",0
439453,46,63,south quay square,12,street_name,94087,"storage 17, discovery dock apartments east, 3 ...",1
439451,65,72,london,3,city,94087,"storage 17, discovery dock apartments east, 3 ...",0


In [16]:
# all_entities = pd.json_normalize(all_entities_json, record_path = "labels",
#                        meta = ['datapoint_id', 'text'])

all_entities['label_id_count'] = all_entities.groupby(['datapoint_id', 'label']).cumcount()

all_entities['label_text'] =all_entities['label_text'].str.replace("100-1124", "100-112")

all_entities.drop(columns = ['data_stuff.data', 'groundTruthId', 'labellingFunctionId'], inplace = True)

## Example of the data frame of labels

In [20]:
all_entities[all_entities['label_text'].str.contains('miller way, 15-25')]

,start,end,text,labelId,label,datapoint_id,label_text,label_id_count
64,0,4,2-24,13,street_number,13,"2-24 (even) miller way, 15-25 hammonds drive (...",0
69,6,10,even,16,number_filter,13,"2-24 (even) miller way, 15-25 hammonds drive (...",0
59,12,22,miller way,12,street_name,13,"2-24 (even) miller way, 15-25 hammonds drive (...",0
61,12,22,miller way,12,street_name,13,"2-24 (even) miller way, 15-25 hammonds drive (...",1
65,24,29,15-25,13,street_number,13,"2-24 (even) miller way, 15-25 hammonds drive (...",1
67,46,49,odd,16,number_filter,13,"2-24 (even) miller way, 15-25 hammonds drive (...",1
63,55,56,1,13,street_number,13,"2-24 (even) miller way, 15-25 hammonds drive (...",2
66,55,59,1-19,13,street_number,13,"2-24 (even) miller way, 15-25 hammonds drive (...",3
68,72,75,odd,16,number_filter,13,"2-24 (even) miller way, 15-25 hammonds drive (...",2
60,77,84,fengate,12,street_name,13,"2-24 (even) miller way, 15-25 hammonds drive (...",2


## Identify multi versus single address observations

Some addresses may have the form xx to yy but should not be expanded as this is a building that covers multiple street numbers. Items such as these need to be carefully removed before expansion

In [46]:
xx_to_yy_regex = r'^\d+(\s)?(-|to)(\s)?\d+$'

multi_check_df = all_entities[['datapoint_id', 'text', ]].drop_duplicates()
multi_check_df['comma_count'] = multi_check_df['text'].str.count(',')
multi_check_df['land'] = multi_check_df['text'].str.contains(r"^(land|plot|airspace|car|parking)", case = False)

multi_check_df['business'] = multi_check_df['text'].str.contains(r"(cinema)|(hotel)|(office)|(\bpub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(\bstore\b)|(\bstorage\\b)|(company)|(ltd)|(limited)|(plc)", case = False)
temp_df = all_entities[['datapoint_id', 'label']].groupby(['datapoint_id', 'label']).value_counts().to_frame(name = "counts").reset_index().pivot(index = 'datapoint_id', columns = 'label', values = 'counts').fillna(0)
#test['datapoint_id'] = test.index

xx_to_yy_street_counts = all_entities['datapoint_id'][all_entities['label_text'].str.contains(
    xx_to_yy_regex)& (all_entities['label']=="street_number")
                            ].to_frame(name = 'datapoint_id').groupby('datapoint_id').size().to_frame(name = 'xx_to_yy_street_counts')

xx_to_yy_unit_counts = all_entities['datapoint_id'][all_entities['label_text'].str.contains(
    xx_to_yy_regex)& (all_entities['label']=="unit_id")
                            ].to_frame(name = 'datapoint_id').groupby('datapoint_id').size().to_frame(name = 'xx_to_yy_unit_counts')

multi_check_df = multi_check_df.merge(temp_df, how = 'left', left_on = "datapoint_id", right_index = True).\
merge(xx_to_yy_street_counts, how = 'left', left_on = "datapoint_id", right_index = True).\
merge(xx_to_yy_unit_counts, how = 'left', left_on = "datapoint_id", right_index = True).fillna(0)


del xx_to_yy_street_counts
del xx_to_yy_unit_counts

#separate the classes using logical rules
multi_check_df['class'] = np.select(
    [
        multi_check_df['land']== True,
        multi_check_df['business']== True,
        (multi_check_df['building_name']==1) & (multi_check_df['unit_id'] == 0), #this has to go infront of 'multi_check_df['xx_to_yy_unit_counts']>0'
        multi_check_df['xx_to_yy_unit_counts']>0,
        multi_check_df['street_number']>1,
        multi_check_df['unit_id']>1,
        (multi_check_df['street_number']<=1) & (multi_check_df['xx_to_yy_street_counts']<=1) & (multi_check_df['unit_id']<=1) ##This does most of the heavy lifting
    ], 
    [
        'single',
        'single',
        'single',
        'multi',
        'multi',
        'multi',
        'single',
        
    ], 
    default='unknown'
)
multi_check_df


<ipython-input-46-c0e57ee51fa8>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  multi_check_df['land'] = multi_check_df['text'].str.contains(r"^(land|plot|airspace|car|parking)", case = False)
<ipython-input-46-c0e57ee51fa8>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  multi_check_df['business'] = multi_check_df['text'].str.contains(r"(cinema)|(hotel)|(office)|(\bpub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(\bstore\b)|(\bstorage\\b)|(company)|(ltd)|(limited)|(plc)", case = False)
<ipython-input-46-c0e57ee51fa8>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  xx_to_yy_street_counts = all_entities['datapoint_id'][all_entities['label_text'].str.contains(
<ipython-input-46-c0e57ee51fa8>:15: UserWarning: Th

,datapoint_id,text,comma_count,land,business,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,xx_to_yy_street_counts,xx_to_yy_unit_counts,class
1,0,westleigh lodge care home,0,False,True,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,single
0,0,nel pan lane,0,False,False,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,single
2,0,leigh,0,False,False,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,single
4,0,wn7 5jt,0,False,False,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,single
8,1,flat,0,False,False,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439452,94087,storage,0,False,False,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,single
439450,94087,3,0,False,False,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,single
439449,94087,south quay square,0,False,False,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,single
439451,94087,london,0,False,False,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,single


### Get the indexes of multi properties

In [47]:
multi_unit_id = set(multi_check_df['datapoint_id'][(multi_check_df['class']=='multi') &( multi_check_df['unit_id']>0)].tolist())
multi_property = set(multi_check_df['datapoint_id'][(multi_check_df['class']=='multi') &( multi_check_df['unit_id']==0)].tolist())
all_multi_ids = list(multi_unit_id) +list(multi_property)
multi_check_df.groupby('class').size()

class
multi      63563
single    353397
dtype: int64

# checking weird regex problems

In [9]:
number_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]



#these are just tests for the expansion functions

multi_id_string = "1to30"
number_list = expand_multi_id(multi_id_string)
print(number_list)
correct_numbers_even = filter_contiguous_numbers(number_list, "even")
correct_numbers_odd = filter_contiguous_numbers(number_list, "odd")
correct_numbers_all = filter_contiguous_numbers(number_list, None)
print("original list" + str(number_list), "\neven list", str(correct_numbers_even),
     "\nodd list", str(correct_numbers_odd),
     "\nall numbers", str(correct_numbers_all))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
original list[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] 
even list [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30] 
odd list [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29] 
all numbers [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [10]:
all_entities['label_text'].to_list()[0]

'westleigh lodge care home'

## Spread label data

In [48]:
#pivot the columns so that each label class is it's own column and the value in the column is the text

temp_df = all_entities[all_entities.datapoint_id.isin(all_multi_ids)]

temp_df['index'] = temp_df.index
df = temp_df[['index', 'label', 'label_text']].pivot(index='index',columns='label',values='label_text')
#add the datapoint_id back in for each of joining
df = pd.concat([temp_df['datapoint_id'], df], axis=1).merge(temp_df[['datapoint_id' ,'text']].drop_duplicates(), 
          how = "left",
          left_on = "datapoint_id", right_on = "datapoint_id")
del temp_df

<ipython-input-48-8bdee3a0bd7d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['index'] = temp_df.index


In [51]:
df[df.street_name.str.contains('\(even\) miller way')==True].drop_duplicates()#.to_csv('/tf/empty_homes_data/delete_me.csv')

,datapoint_id,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,text
18,13,block,NaN,NaN,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,2-24
19,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,even
20,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,miller way
21,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,15-25
22,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,odd
23,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,1
24,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,1-19
25,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,fengate
26,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,peterborough
27,13,block,NaN,block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,2-24


## Add blockers

Blockers prevent the filling of wrong information. As an example if a building is going to back fill up previous addresses it should not back fill past another street as this is highly unlikely to be the same building

In [50]:
df['building_name'][df['street_name'].notnull()] = 'block'
df['street_number'][df['street_name'].notnull()] = 'block' #for multi-flats inside a common building

#returns true if current number filter is null and the next row has street_number or unit id is not null
#prevents number filters propergsating back across roads and unit ids
number_filter_block = df['number_filter'].isnull() & (df['street_number'].shift().notnull() |df['unit_id'].shift().notnull())
df['number_filter'][number_filter_block] = 'block'

## Backfill 

Backfilling adds address information in. However, street address should only be back filled for multi addresses.
I need to work out how to do flat, which may be before or after the unit ID
Also I don't think this is a very good way of doing it at all. Using the pre-spread list is probably better and only working on the multi-addresses is probably a much faster and cleaner way. But I will have to think about how to do it

In [30]:
df['number_filter'] = df[['datapoint_id','number_filter']].groupby('datapoint_id').fillna(method ='bfill')
df['building_name'] = df[['datapoint_id','building_name']].groupby('datapoint_id').fillna(method ='bfill')
df['street_number'] = df[['datapoint_id','street_number']].groupby('datapoint_id').fillna(method ='bfill')
df['postcode'] = df[['datapoint_id','postcode']].groupby('datapoint_id').fillna(method ='bfill')
df['street_name'] = df[['datapoint_id','street_name']].groupby('datapoint_id').fillna(method ='bfill')
df['number_filter'] = df[['datapoint_id','number_filter']].groupby('datapoint_id').fillna(method ='bfill')
df['city'] = df[['datapoint_id','city']].groupby('datapoint_id').fillna(method ='bfill')
df['unit_type'] = df[['datapoint_id','unit_type']].groupby('datapoint_id').fillna(method ='bfill')

In [31]:
df

,datapoint_id,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,text
0,13,block,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,miller way
1,13,block,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,fengate
2,13,block,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,1
3,13,block,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,2-24
4,13,block,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,"2-24 (even) miller way, 15-25 hammonds drive (...",block,NaN,NaN,15-25
...,...,...,...,...,...,...,...,...,...,...
680067,94085,NaN,NaN,NaN,"9th and 10th floors, 52 lime street, london (e...",NaN,NaN,NaN,NaN,10
680068,94085,NaN,NaN,NaN,"9th and 10th floors, 52 lime street, london (e...",NaN,NaN,NaN,NaN,9t
680069,94085,NaN,NaN,NaN,"9th and 10th floors, 52 lime street, london (e...",NaN,NaN,NaN,NaN,52
680070,94085,NaN,NaN,NaN,"9th and 10th floors, 52 lime street, london (e...",NaN,NaN,NaN,NaN,london


In [32]:
expanded_street = df[df.datapoint_id.isin(multi_property) & df.street_number.str.contains(xx_to_yy_regex)].reset_index()
expanded_unit_id = df[df.datapoint_id.isin(multi_unit_id) & df.unit_id.str.contains(xx_to_yy_regex)].reset_index()

expanded_street = expand_dataframe_numbers(expanded_street, column_name = "street_number" )
expanded_unit_id = expand_dataframe_numbers(expanded_unit_id, column_name = "unit_id" )

<ipython-input-32-6dd2bb121964>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_street = df[df.datapoint_id.isin(multi_property) & df.street_number.str.contains(xx_to_yy_regex)].reset_index()
<ipython-input-32-6dd2bb121964>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_unit_id = df[df.datapoint_id.isin(multi_unit_id) & df.unit_id.str.contains(xx_to_yy_regex)].reset_index()


ValueError: No objects to concatenate

In [17]:
#unit id and street number that does does not have the xx to yy format and so has already been expanded by spreaing and backfilling
expanded_street_simple = df[df.datapoint_id.isin(multi_property) & (df.street_number.str.contains(xx_to_yy_regex)==False) & (df.street_number!='block')].reset_index()
expanded_unit_id_simple = df[df.datapoint_id.isin(multi_unit_id) & (df.unit_id.str.contains(xx_to_yy_regex)==False) & (df.unit_id!='block')].reset_index()

<ipython-input-17-53d39f72e523>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_street_simple = df[df.datapoint_id.isin(multi_property) & (df.street_number.str.contains(xx_to_yy_regex)==False) & (df.street_number!='block')].reset_index()
<ipython-input-17-53d39f72e523>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_unit_id_simple = df[df.datapoint_id.isin(multi_unit_id) & (df.unit_id.str.contains(xx_to_yy_regex)==False) & (df.unit_id!='block')].reset_index()


In [18]:
#pivot the columns so that each label class is it's own column and the value in the column is the text

#remove the multi-addresses
single_address_only =all_entities[~all_entities['datapoint_id'].isin(all_multi_ids)]
#remove all but the first instance of a label in the remaining instances
#this is because for single addresses there should be only a single label for each class
single_address_only =single_address_only[single_address_only['label_id_count']==0]
df2 = single_address_only.pivot(index='datapoint_id',columns='label',values='label_text')
#add the datapoint_id back in for each of joining
df2 = df2.merge(single_address_only[['datapoint_id', 'text']].drop_duplicates(), 
          how = "left",
          left_on = "datapoint_id", right_on = "datapoint_id")

df2

/usr/local/lib/python3.8/dist-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,datapoint_id,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,text
0,0,westleigh lodge care home,leigh,NaN,wn7 5jt,nel pan lane,NaN,NaN,NaN,"westleigh lodge care home, nel pan lane, leigh..."
1,1,NaN,manchester,NaN,m1 3he,canal street,1a,1,flat,"flat 1, 1a canal street, manchester (m1 3he)"
2,2,NaN,manchester,NaN,m3 4ay,regent road,1,201,flat,"flat 201, 1 regent road, manchester (m3 4ay)"
3,3,NaN,wigan,NaN,wn4 9aa,gerard street,NaN,NaN,land,"land at 2a gerard street, ashton in makerfield..."
4,4,NaN,manchester,NaN,m15 4nz,worsley street,NaN,111,unit,"unit 111, timber wharf, worsley street, manche..."
...,...,...,...,...,...,...,...,...,...,...
87567,94081,heritage tower,london,NaN,e14 3nw,east ferry road,118,807,flat,"flat 807, heritage tower, 118 east ferry road,..."
87568,94082,ormond house,london,NaN,ec4n 4ua,queen victoria street,4t,NaN,NaN,"4th floor, ormond house, 63 queen victoria str..."
87569,94084,NaN,london,NaN,ec2m 3ad,bishopsgate,155,NaN,NaN,"155 bishopsgate, london (ec2m 3ad)"
87570,94086,NaN,london,NaN,ec3a 7ba,NaN,NaN,NaN,NaN,"part of tenth floor, 6 bevis marks, london (ec..."


In [19]:
full_expanded_data = pd.concat([expanded_street, 
           expanded_unit_id, 
           expanded_street_simple, 
           expanded_unit_id_simple, 
           df2, ])

In [20]:
ocod_data =  pd.read_csv('/tf/empty_homes_data/' +
                    'OCOD_FULL_2022_02.csv',
                   encoding_errors= 'ignore').rename(columns = lambda x: x.lower().replace(" ", "_"))
#empty addresses cannot be used. however there are only three so not a problem
ocod_data = ocod_data.dropna(subset = 'property_address')
ocod_data.reset_index(inplace = True, drop = True)
ocod_data = ocod_data[['title_number', 'tenure', 'district', 'county',
       'region', 'multiple_address_indicator', 'price_paid', 'property_address']]

<ipython-input-20-83c4912c1574>:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  ocod_data =  pd.read_csv('/tf/empty_homes_data/' +


In [21]:
full_expanded_data = full_expanded_data.merge(ocod_data, how = "left", left_on = "datapoint_id", right_index = True)
full_expanded_data['property_address'].str.lower().equals(full_expanded_data['text']) #This shows the match works

True

In [22]:
full_expanded_data['property_address'].str.lower().equals(full_expanded_data['text']) #This shows the match works

True

In [23]:
full_expanded_data['within_title_id'] = full_expanded_data.groupby('title_number').cumcount()+1
full_expanded_data['unique_id'] = [str(x) + '-' + str(y) for x, y in zip(full_expanded_data['title_number'], full_expanded_data['within_title_id'])]

tmp_df =((full_expanded_data[['title_number', 'within_title_id']].groupby('title_number').max('within_title_id'))>1)
tmp_df.columns = tmp_df.columns.str.replace('within_title_id', 'within_larger_title') #could also be called nested_address
full_expanded_data = full_expanded_data.merge(tmp_df, how = "left", left_on = "title_number", right_index = True)


full_expanded_data['postcode'] =full_expanded_data['postcode'].str.upper()
del tmp_df

#re-order the columns and drop columns that are not needed

full_expanded_data =full_expanded_data[['title_number', 'within_title_id', 'unique_id', 'within_larger_title',  'tenure','unit_id', 'unit_type','building_name','street_number', 'street_name', 'postcode','city',  'district', 'county', 'region',
       'multiple_address_indicator', 'price_paid' ,'property_address']].replace('block', np.NaN)

#save as CSV
full_expanded_data.to_csv("/tf/empty_homes_data/OCOD_cleaned_expanded.csv")

In [ ]:
full_expanded_data[full_expanded_data['street_name'].isnull()]['property_address'].to_csv('/tf/empty_homes_data/street_is_null.csv')

In [ ]:
full_expanded_data[full_expanded_data.within_title_id==full_expanded_data.within_title_id.max()].reset_index()['property_address'][0]


In [ ]:
300000*700